In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
# Create a set of random latitude and longitude combinations 
lats = np.random.uniform(-90, 90, size = 2000)
lngs = np.random.uniform(-180, 180, size = 2000)
lat_lngs = zip(lats, lngs)

In [3]:
# Adding latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
city_name = []
country_code = []

# Dependency
from citipy import citipy

# Iterate through each of coordinate, and find its nearest city and country code. 
# This code will only record one city when there are duplicates. 
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    if city not in city_name:
        city_name.append(city)
        country_code.append(citipy.nearest_city(coordinate[0], coordinate[1]).country_code)
    

In [5]:
# |TESTING TESTING TESTING TESTING TESTING           |
# |__________________________________________________|

# Import request library
import requests 

# Import the API key
from config import weather_api_key

base_url = "http://api.openweathermap.org/data/2.5/weather?q="

city_url = f"{base_url}{city_name[0]}&units=metric&appid={weather_api_key}"

response = requests.get(city_url).json()
#pprint(response)

# Latitude and longitude
lng = response["coord"]["lon"]
lat = response["coord"]["lat"]

# Maximum temperature
max_temp = response["main"]["temp_max"]

# Percent humidity
humidity = response["main"]["humidity"]

# Percent cloudiness
cloudiness = response["clouds"]["all"]

# Wind speed
speed = response["wind"]["speed"]

# Weather description (for example, clouds, fog, light rain, clear sky)
description = response["weather"][0]["description"]

# Print to ensure if all data were pulled
print(f"{lng}, {lat}, {max_temp}, {humidity}, {cloudiness}, {speed}, {description}")

# ---------------------------------------------------
response

10.9333, 9.9833, 21.09, 89, 82, 2.75, broken clouds


{'coord': {'lon': 10.9333, 'lat': 9.9833},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 21.09,
  'feels_like': 21.58,
  'temp_min': 21.09,
  'temp_max': 21.09,
  'pressure': 1012,
  'humidity': 89,
  'sea_level': 1012,
  'grnd_level': 952},
 'visibility': 10000,
 'wind': {'speed': 2.75, 'deg': 212, 'gust': 7.89},
 'clouds': {'all': 82},
 'dt': 1632616425,
 'sys': {'country': 'NG', 'sunrise': 1632632706, 'sunset': 1632676201},
 'timezone': 3600,
 'id': 2324857,
 'name': 'Pindiga',
 'cod': 200}

In [6]:
# Creating a dataframe. 
df = pd.DataFrame()
df["City"] = city_name
df["Country"] = country_code
df["Lat"] = ""
df["Lng"] = ""
df["Max Temp"] = ""
df["Humidity"] = ""
df["Cloudiness"] = ""
df["Wind Speed"] = ""
df["Current Description"] = ""


In [7]:
# Iterration begins here

base_url = "http://api.openweathermap.org/data/2.5/weather?q="

for index,row in df.iterrows():
    try:
        city = row["City"].capitalize()
        row["City"] = city
        
        country = row["Country"].upper()
        row["Country"] = country
        
        url = f"{base_url}{city}&units=metric&appid={weather_api_key}"
        response = requests.get(url).json()
        df.loc[index, "Lng"] = response["coord"]["lon"]
        df.loc[index, "Lat"] = response["coord"]["lat"]
        df.loc[index, "Max Temp"] = response["main"]["temp_max"]
        df.loc[index, "Humidity"] = response["main"]["humidity"]
        df.loc[index, "Cloudiness"] = response["clouds"]["all"]
        df.loc[index, "Wind Speed"] = response["wind"]["speed"]
        df.loc[index, "Current Description"] = response["weather"][0]["description"]
    
    except:
        print(f"No result found - Skipping index {index}, city name: {city}, {country}")
        pass

No result found - Skipping index 13, city name: Illoqqortoormiut, GL
No result found - Skipping index 15, city name: Taolanaro, MG
No result found - Skipping index 21, city name: Belushya guba, RU
No result found - Skipping index 54, city name: Sahrak, AF
No result found - Skipping index 86, city name: Kobojango, BW
No result found - Skipping index 87, city name: Samalaeulu, WS
No result found - Skipping index 90, city name: Barentsburg, SJ
No result found - Skipping index 91, city name: Grand river south east, MU
No result found - Skipping index 93, city name: Mochitlan, MX
No result found - Skipping index 101, city name: Blonduos, IS
No result found - Skipping index 113, city name: Amderma, RU
No result found - Skipping index 123, city name: Alotau, PG
No result found - Skipping index 134, city name: Arkalokhorion, GR
No result found - Skipping index 151, city name: Dinsor, SO
No result found - Skipping index 158, city name: Stamsund, NO
No result found - Skipping index 186, city nam

In [8]:
# Fixinng format for lat and lng columns

df["Lat"] = pd.to_numeric(df["Lat"]).map("{:.2f}".format)
df["Lng"] = pd.to_numeric(df["Lng"]).map("{:.2f}".format)

In [9]:
# Exporting new DataFrame as a CSV file. 

to_file = "WeatherPy_Database.csv"
df.to_csv(to_file, index = False)

In [10]:
df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Pindiga,NG,9.98,10.93,21.09,89,82,2.75,broken clouds
1,Surt,LY,31.21,16.59,22.69,67,0,3.81,clear sky
2,Hobart,AU,-42.88,147.33,12.25,59,20,1.79,few clouds
3,Ushuaia,AR,-54.80,-68.30,6.81,61,75,9.77,broken clouds
4,Yellowknife,CA,62.46,-114.35,7.45,77,75,1.79,broken clouds
...,...,...,...,...,...,...,...,...,...
762,Honiara,SB,-9.43,159.95,29.84,70,84,2.78,broken clouds
763,Zima,RU,53.92,102.04,2.6,71,88,1.36,overcast clouds
764,Tecpan,MX,14.76,-90.99,16.62,96,99,1.29,overcast clouds
765,Mutare,ZW,-18.97,32.67,12.61,66,0,1.14,clear sky
